In [1]:
pip install torch transformers

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Setup the Model ID
# Qwen2.5-0.5B-Instruct is a very capable small model (approx 0.5B parameters)
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

print(f"Loading {model_id}...")

# 2. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"   # Uses GPU if available, otherwise CPU
)

print("\n--- Model Loaded. Type 'quit' to exit. ---\n")

# 3. Chat Loop
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break

    # Append user message
    messages.append({"role": "user", "content": user_input})

    # Prepare inputs using the chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate response
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
    )

    # Decode and extract only the new response
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"Bot: {response}\n")

    # Update history
    messages.append({"role": "assistant", "content": response})

Loading Qwen/Qwen2.5-0.5B-Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


--- Model Loaded. Type 'quit' to exit. ---

You: hi there my name is Gul, tell me about yourself


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: Hello! As an AI language model, I don't have personal names or experiences like humans do. However, I can help you with any questions or tasks that you may have to the best of my abilities based on the information available to me. Is there anything specific you would like to know or discuss?

You: tell me who wrote the great gatsby
Bot: "The Great Gatsby" was written by F. Scott Fitzgerald. He was an American novelist and short story writer who lived from 1896 to 1940. The novel was first published in 1925 and has since become one of the most famous works in literature. It follows the life of Jay Gatsby, a wealthy man who has fallen in love with Daisy Buchanan, the daughter of the title character's father. The novel explores themes such as money, love, and illusion, among others.

You: quit


In [3]:
pip install edge-tts

In [5]:
import torch
import asyncio
import edge_tts
import os
from transformers import pipeline

# 1. SETUP MODEL (If not already loaded in your interactive session)
# If you are pasting this into a running session, skip this block.
model_id = "Qwen/Qwen2.5-1.5B-Instruct"
print(f"Loading {model_id} for script writing...")
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 2. GENERATE THE SCRIPT
def generate_script(topic):
    print(f"\nThinking of a discussion about: {topic}...\n")

    # We explicitly tell the model how to format the text so we can parse it later.
    system_prompt = (
        "You are a scriptwriter. Write a lively 2-minute dialogue between two friends, "
        "Alice and Bob. They are debating the topic provided. "
        "Strictly follow this format:\n"
        "Alice: [Text]\n"
        "Bob: [Text]\n"
        "Alice: [Text]\n"
        "... and so on. Do not add scene descriptions or actions *like this*, only dialogue."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Topic: {topic}"}
    ]

    outputs = pipe(
        messages,
        max_new_tokens=1024, # Long enough for a 2-3 min convo
        do_sample=True,
        temperature=0.7,
    )

    script_text = outputs[0]["generated_text"][-1]["content"]
    return script_text

# 3. CONVERT TO AUDIO (Async function required for edge-tts)
async def create_podcast(script_text):
    lines = script_text.split('\n')
    audio_files = []

    # Define Voices
    # Alice = Female US Voice, Bob = Male US Voice
    voice_alice = "en-US-AriaNeural"
    voice_bob = "en-US-GuyNeural"

    print("Generating audio segments...")

    for i, line in enumerate(lines):
        if ":" not in line:
            continue # Skip lines that aren't dialogue

        speaker, text = line.split(":", 1)
        text = text.strip()

        # Select Voice
        if "Alice" in speaker:
            voice = voice_alice
        elif "Bob" in speaker:
            voice = voice_bob
        else:
            continue

        # Generate Audio for this line
        filename = f"temp_{i:03d}.mp3"
        communicate = edge_tts.Communicate(text, voice)
        await communicate.save(filename)
        audio_files.append(filename)
        print(f"Generated line {i}: {speaker}")

    # 4. MERGE FILES (Simple Binary Concatenation)
    print("Merging audio files...")
    with open("final_podcast.mp3", "wb") as outfile:
        for fname in audio_files:
            with open(fname, "rb") as infile:
                outfile.write(infile.read())
            # Clean up temp file
            os.remove(fname)

    print("\nSUCCESS! Saved as 'final_podcast.mp3'")

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # Get user topic
    topic = input("Enter a topic for the discussion (e.g., 'The future of AI'): ")

    # 1. Generate Text
    script = generate_script(topic)
    print("\n--- Generated Script ---\n")
    print(script)
    print("\n------------------------\n")

    # 2. Generate Audio
    await create_podcast(script)

Loading Qwen/Qwen2.5-1.5B-Instruct for script writing...


Device set to use cuda:0


Enter a topic for the discussion (e.g., 'The future of AI'): youth of Pakistan in the eyes of elderlies 

Thinking of a discussion about: youth of Pakistan in the eyes of elderlies ...


--- Generated Script ---

Alice: Oh my goodness, I can't believe how much things have changed since I was younger! The internet is everywhere now, and we're always connected to each other.

Bob: Yeah, it's crazy how technology has advanced so quickly. But I remember when people used to rely on phones for everything. It felt like magic then!

Alice: And let's talk about fashion. Back in my day, you had to be really careful with your clothes because they were expensive. Now, anyone can afford designer brands.

Bob: That's true. But do you think there's still something special about being young?

Alice: Absolutely! Young people bring energy and new ideas to society. Plus, who doesn't love the thrill of adventure?

Bob: Well, I guess that's part of what makes life exciting too. But sometimes I miss the sim